<a href="https://colab.research.google.com/github/rkhoury1/MachineLearningShenaniganz/blob/main/Khoury_Ramie_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from datetime import datetime, date, time, timezone
import pandas as pd
import numpy as np
import string
import glob
import os
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk

***1. Predicting Stock Price Movement***

***b) i) Stock Twits Data***

Open and Create Dataset

In [ ]:
file_list = glob.glob(os.path.join(os.getcwd(), "twits", "*.txt"))
df_all = {}
for file_path in file_list:
    with open(file_path) as json_file:
        temp = json.load(json_file)
    name = "".join(list(file_path)[51:]).strip('.txt')
    df_all[name] = temp

In [ ]:
df_all.keys()

dict_keys([])

------- Using just one dataset -------

In [ ]:
with open('twits/AXP.txt') as json_file:
    axp = json.load(json_file)

FileNotFoundError: ignored

In [ ]:
data = []
for i in range(0, len(axp)):
    temp = []
    date = axp[i]['created_at']
    date = ''.join(list(date)[5:16])
    date = datetime.strptime(date, '%d %b %Y').date()
    temp.append(date)
    if axp[i]['sentiment'] != None:
        temp.append(axp[i]['sentiment']['class'])
    else:
        temp.append('None')
    data.append(temp)

In [ ]:
df_axp = pd.DataFrame(data, columns = ['Date', 'Sentiment'])
df_axp.head(5)

Moving Average for Polarity (St)

In [ ]:
df_new = pd.DataFrame(df_axp.groupby(['Date', 'Sentiment'])['Sentiment'].count())
df_new.columns = ['Count']
df_new.reset_index(inplace=True)
df_new.iloc[0:10]

In [ ]:
#get list of dates
dates = []
for i in range(0,len(df_new)):
    if df_new.iloc[i]['Date'] not in dates:
        dates.append(df_new.iloc[i]['Date'])
len(dates)

In [ ]:
polarity = []
sentiments = ['bullish', 'bearish', 'None']
for k in range(0,len(dates)):
    dictsent = {}
    temp = df_new[df_new['Date'] == dates[k]]
    for i in sentiments:
        if i in list(temp['Sentiment']):
            dictsent[i] = int(temp[temp['Sentiment'] == i]['Count'])
        else:
            dictsent[i] = 0
    dictsent
    polarity.append((dictsent['bullish']-dictsent['bearish'])/sum(dictsent.values()))

In [ ]:
st = []
for i in range(0, len(polarity)):
    if i == 0:
        st.append(None)
    elif i == len(polarity)-1:
        st.append(None)
    else:
        add = polarity[i-1]+polarity[i]+polarity[i+1]
        st.append(add/3)

Volume

In [ ]:
df_new = pd.DataFrame(df_axp.groupby('Date').count())
df_new.reset_index(level=0, inplace=True)
df_new.sort_values(by=['Date'])
df_new.columns = ['Date', 'Count']
volume = list(df_new['Count'])

Volume Change (MV1,t)

In [ ]:
mv1t = []
for index, row in df_new.iterrows():
    if index == 0:
        mv1t.append(None)
    else:
        mv1t.append((row['Count']-temp)/temp)
    temp = row['Count']
print(len(mv1t))

Average Volume Change(MV10,t)

In [ ]:
mv10t = []
for i in range(0, len(df_new)):
    if i <= 9:
        mv10t.append(None)
    else:
        temp = 0
        for k in range(i-10, i):
            temp = temp + df_new.iloc[k]['Count']
        mv10t.append(df_new.iloc[i]['Count']/temp)

In [ ]:
len(mv10t)

In [ ]:
axp_twit = pd.DataFrame({
    'Date':dates, 
    'Polarity':polarity,
    'St': st,
    'Volume': volume,
    'MV1t': mv1t,
    'MV10t:': mv10t})
axp_twit

***ii) Price Data***

In [ ]:
axp_price = pd.read_csv('csv/AXP.csv')

In [ ]:
#focus only on closing
axp_price = axp_price[['Date', 'CLOSE']].sort_values('Date')
axp_price

***iii) Prediction Target***

In [ ]:
day3return = []
close = list(axp_price['CLOSE'])
for i in range(0, len(close)):
    if i < len(close)-3:
        temp = (close[i+3]-close[i])/close[i]
        day3return.append(temp)
    else:
        day3return.append(None)

In [ ]:
day5return = []
close = list(axp_price['CLOSE'])
for i in range(0, len(close)):
    if i < len(close)-5:
        temp = (close[i+5]-close[i])/close[i]
        day5return.append(temp)
    else:
        day5return.append(None)

In [ ]:
axp_price['3DayReturn'] = day3return
axp_price['5DayReturn'] = day5return
axp_price

***c) Pre-Processing and Exploratory***

In [ ]:
df_all.keys()

In [ ]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
             "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
             'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
             'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 
             'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
             'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
             'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
             'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 
             'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
             'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 
             't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 
             've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
             "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', 
             "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
def checktag(string):
    string = list(string)
    count = 0
    for i in string:
        if i == '$':
            count = count+1
    if count > 2:
        return True
    else:
        return False

In [ ]:
doc = []
date = []
temp = df_all['AXP']
for i in range(0, len(temp)):
    if checktag(temp[i]['body']) == False:
        words = temp[i]['body'].split(' ')
        doc.append(words)
        dat = "".join(list(temp[3]['created_at'])[5:16])
        date.append(datetime.strptime(dat, '%d %b %Y').date())
len(doc)

In [ ]:
words = temp[20]['body'].split(' ')
words

In [ ]:
checktag(temp[20]['body'])

In [ ]:
for i in list(temp[10]['body']):
    if 